# Tweets Preprocessing
Damit wir eine ordentliche Auswertung der Twitter-Daten machen können müssen wir die Date zunächst mittel dem Text-Preprocessing bereinigen.

Erstes Laden der benötigten Bibliotheken

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Daten laden
Wir holen uns unsere Daten der beiden Bundesländer.
Damit wir den Vorgang nicht zweimal machen müssen hängen wir beide DataFrame zusammen. Um die beiden Datensätze später wieder ordentlich trennen zu können erstellen wir eine Spalte mit Label (1/0) für die jeweilige Himmelsrichtung.

In [2]:
df_westen=pd.read_csv('tweets_westen.zip')
df_osten=pd.read_csv('tweets_osten.zip')

In [3]:
df_westen['Label']='1'
df_osten['Label']='0'

Wenn wir uns den DataFrame jetzt anschauen, sehen wir, dass die Daten am Anfang (West) Label=1 und die Daten am Ende (Ost) Label=0 haben.

In [4]:
df=pd.concat([df_westen, df_osten], axis=0)
df.head()

,Name,Datum,Inhalt,Label
0,@RegierungBW,2022-09-22 07:45:03+00:00,@Schwabenhelga natürlich hat nicht jeder das G...,1
1,@RegierungBW,2022-09-22 07:31:26+00:00,In manchem Keller verstecken sich wahre #Strom...,1
2,@RegierungBW,2022-09-22 06:59:09+00:00,Auf unserem #Beteiligunsgsportal können Sie si...,1
3,@RegierungBW,2022-09-21 11:52:04+00:00,@martinruff1 @UmweltBW Konkrete Ausbauziele fü...,1
4,@RegierungBW,2022-09-21 11:33:38+00:00,Wir feiern unser #Juhubiläum mit #70Jahre70Köp...,1


In [5]:
df.tail()

,Name,Datum,Inhalt,Label
10960,@ThuerLandtag,2018-08-21 08:38:04+00:00,RT @LPK_Thueringen: 1/1 LPK-Jour Fixe mit den ...,0
10961,@ThuerLandtag,2018-08-21 08:38:00+00:00,RT @LPK_Thueringen: 1/2 LPK-Jour Fixe mit den ...,0
10962,@ThuerLandtag,2018-08-21 07:09:01+00:00,"@PaparazzoOrange Ja, die Dr.S. 6/5479 fehlte w...",0
10963,@ThuerLandtag,2018-08-21 06:36:40+00:00,"@PaparazzoOrange Hm, ja, das ist ja schon etwa...",0
10964,@ThuerLandtag,2018-08-20 14:21:31+00:00,RT @TKinTH: Mit dabei sind Präsident @ThuerLan...,0


# Daten-Preprocessing

Der Preprocessing-Vorgang wurde eher händisch gemacht und nicht mit Spacy, da es hier erhebliche Probleme mit dem Datenformat PdDf gab.
geklappt hat es mit folgender Lösungsvorlage: https://practicaldatascience.co.uk/machine-learning/how-to-preprocess-text-for-nlp-in-four-easy-steps.

Grundsätzlich: Für die meisten Schritte wird zunächst eine Funktion entworfen, die dem jeweiligem Preprocessing-Schritt entspricht. Danach wird eine neue Spalte erstellt, deren Inhalt dann dem Inhalt der vorherigen Spalte entspricht, auf welchem aber die Funktion angewandt wurde.

# 1. Lower-Case


In [6]:
df['Inhalt']=df['Inhalt'].str.lower()

In [7]:
df.head

<bound method NDFrame.head of                 Name                      Datum  \
0       @RegierungBW  2022-09-22 07:45:03+00:00   
1       @RegierungBW  2022-09-22 07:31:26+00:00   
2       @RegierungBW  2022-09-22 06:59:09+00:00   
3       @RegierungBW  2022-09-21 11:52:04+00:00   
4       @RegierungBW  2022-09-21 11:33:38+00:00   
...              ...                        ...   
10960  @ThuerLandtag  2018-08-21 08:38:04+00:00   
10961  @ThuerLandtag  2018-08-21 08:38:00+00:00   
10962  @ThuerLandtag  2018-08-21 07:09:01+00:00   
10963  @ThuerLandtag  2018-08-21 06:36:40+00:00   
10964  @ThuerLandtag  2018-08-20 14:21:31+00:00   

                                                  Inhalt Label  
0      @schwabenhelga natürlich hat nicht jeder das g...     1  
1      in manchem keller verstecken sich wahre #strom...     1  
2      auf unserem #beteiligunsgsportal können sie si...     1  
3      @martinruff1 @umweltbw konkrete ausbauziele fü...     1  
4      wir feiern unser #juhubil

# 2. Tokenize

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Die folgende Funktion nimmt eine Spalte (Text) entgegen und wandelt jedes Element in ein Token um und gibt sie als Inhalt einer Liste wieder.

In [9]:
def tokenize(column):
  tokens=nltk.word_tokenize(column)
  return[w for w in tokens if w.isalpha()]

Wir übergeben der Funktion mit lambda eine Spalte (Inhalt) und führen die Funktion aus. Das Ergebnis wird in einer neuen Spalte gespeichert.

In [10]:
df['tokenized']=df.apply(lambda x: tokenize(x['Inhalt']), axis=1)
df[['Inhalt', 'tokenized']].head

<bound method NDFrame.head of                                                   Inhalt  \
0      @schwabenhelga natürlich hat nicht jeder das g...   
1      in manchem keller verstecken sich wahre #strom...   
2      auf unserem #beteiligunsgsportal können sie si...   
3      @martinruff1 @umweltbw konkrete ausbauziele fü...   
4      wir feiern unser #juhubiläum mit #70jahre70köp...   
...                                                  ...   
10960  rt @lpk_thueringen: 1/1 lpk-jour fixe mit den ...   
10961  rt @lpk_thueringen: 1/2 lpk-jour fixe mit den ...   
10962  @paparazzoorange ja, die dr.s. 6/5479 fehlte w...   
10963  @paparazzoorange hm, ja, das ist ja schon etwa...   
10964  rt @tkinth: mit dabei sind präsident @thuerlan...   

                                               tokenized  
0      [schwabenhelga, natürlich, hat, nicht, jeder, ...  
1      [in, manchem, keller, verstecken, sich, wahre,...  
2      [auf, unserem, beteiligunsgsportal, können, si...  
3      [umwel

# 3. Stopwords-Remove

Dieser Schritt ist für unsere Zwecke am aufwendigsten.\
*Möglich, dass es hierfür einen einfacheren Weg gibt, aber das ist die Lösung, die für uns sehr gut passt.*

In [11]:
nltk. download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Um später eine ordentliche Auswertung der Topics zu bekommen müssen hier einige Wörter herausgefiltert werden. Dazu gehören z.B. Wochentage, Uhrzeiten, Monate (Wahlmonate), alle Namen der Landtagspräsident*innen, Parteien, Bundesländer etc.

Dementsprechend haben wir 3 verschiedene Stopwordlisten.
1. nltk-stopwords
2. custom_stopwords: Wörter, die genauso, wie sie in der Stopwordliste stehen entfernt werden sollen
3. hardstops: Wörter, die so oder so ähnlich entfernt werden sollen.
Beispiel: Wir wollen keine Wörter die etwas mit 'bayern' zu tun haben im DF. Da 'bayern' aber in den verschiedensten Formen (bayern, bayerisch, altbayern) vorkommen kann, filtern wir alle Wörter, die 'bayer' beinhalten. Hierzu benötigen wir eine Liste und eine Funktion.

In [36]:
nltk_stopwords=stopwords.words('german')
custom_stopwords=['plenar','regierung', 'präsident',\
                  'präsidentin', 'abgeordnete', 'plenarsitzung', 'uhr', 'ab', 'wenn'\
                  'geht', 'maximilianeum', 'morgen', 'montag', 'dienstag', 'mittwoch'\
                  'donnerstag', 'freitag', 'samstag', 'sonntag', 'gibt', 'mehr', 'erste', 'zweite',\
                  'mal', 'plenum', 'wurde', 'jahr', 'jahre', 'zeit', 'steht', 'statt', 'november'\
                  'jahren', 'mediakthek', 'ilse', 'aigner', 'barbara', 'stamm', 'muhterem', 'aras', 'dennis', 'buchner', \
                  'ulrike', 'liedtke', 'frank', 'imhoff', 'carola', 'veit', 'astrid', 'wallmann', 'brigit', 'hesse', 'gebriele', \
                  'andretta', 'andré', 'kuper', 'hendrik', 'hering', 'heike', 'becker', 'matthias', 'rößler', 'roeßler', 'gunnar', \
                  'schellenberger', 'kristina', 'herbst', 'birgit', 'pommer', 'amp', 'geht', 'der', 'die', 'das', 'und', 'agh', 'hlt', 'ilseaigner',\
                  'beim' , 'bei']
stopword_liste=nltk_stopwords+custom_stopwords
hardstops=['bayer', 'by', 'rbb', 'baden', 'bawü', 'bw', 'berlin', 'brandenburg', 'bd', 'br', 'bremen', 'hh', 'hamburg',\
             'hessen', 'mecklenburg', 'meckpom', 'mv', 'niedersachsen', 'nordrheinwestfalen', 'nrw', 'rheinlandpfalz', 'rlp', \
             'rp', 'saar', 'sl', 'sachsen', 'sachsen-anhalt', 'anhalt', 'schleswigholstein', 'sh', 'thuer', 'thüringen',\
             'live', 'media', 'stream', 'sitz','uns', 'rt', 'cdu', 'csu', 'afd', 'grüne', 'gruene', 'fdp', 'fw', 'tweet', 'twitter',\
             'service', 'landtag', 'sitzung', 'heut', 'them','htt', 'aktue', 'stund', 'jahr', 'ltnd', 'frakt', 'tag', 'rund' ]

Der Funktion enthalten werden 3 Parameter übergeben: ein wort, die erste Stopwortliste (bestehend aus nltk und custom) und die zweite Stopwortliste.
Die Funktion prüft, ob das zu prüfende Wort zunächst in der Stopwordliste  und und dann, ob es in irgendeiner Form in der hardstops-liste ist.

Folgendes zum Verständnis:
raus=stops=set(stopword_liste)=nltk_stopwordliste+custom_stopwords
hartraus=hardstops

In [37]:
def enthalten(wort,raus,hartraus):
  temp=False
  for x in raus:
    if wort==x:
      temp=True
  for x in hartraus:
    if wort.find(x) != -1:
      temp=True
  return temp

In der zweiten Funktion werden nur Worte wiedergegeben, die nicht in der ersten Funktion gefiltert werden, also alle zu behaltenden Worte.

In [38]:
def remove_stopwords(tokenized_column):
  stops=set(stopword_liste)
  return[word for word in tokenized_column if not enthalten(word,stops,hardstops)]

Wir erstellen als wieder eine neue Spalte, die aus dem Inhalt der vorherigen Spalte basiert, auf welchem die Funktion angewendet wird.

In [39]:
df['stopword_removed']=df.apply(lambda x: remove_stopwords(x['tokenized']), axis=1)
df[['tokenized', 'stopword_removed']].head

<bound method NDFrame.head of                                                tokenized  \
0      [schwabenhelga, natürlich, hat, nicht, jeder, ...   
1      [in, manchem, keller, verstecken, sich, wahre,...   
2      [auf, unserem, beteiligunsgsportal, können, si...   
3      [umweltbw, konkrete, ausbauziele, für, die, ko...   
4      [wir, feiern, unser, juhubiläum, mit, heute, s...   
...                                                  ...   
10960  [rt, fixe, mit, den, thuerlandtag, fraktionsvo...   
10961  [rt, fixe, mit, den, thuerlandtag, fraktionen,...   
10962  [paparazzoorange, ja, die, fehlte, wirklich, d...   
10963  [paparazzoorange, hm, ja, das, ist, ja, schon,...   
10964  [rt, tkinth, mit, dabei, sind, präsident, thue...   

                                        stopword_removed  
0      [schwabenhelga, natürlich, geld, möglichkeit, ...  
1      [keller, verstecken, wahre, stromfresser, heiz...  
2      [online, neunen, klimaschutzgesetz, bürgerbete...  
3      [konkr

# 4. Stemming
Das Stemming sorgt dafür, dass die Anzahl der verschiedenen Formen eines Wortes vereinheitlicht werden und somit weniger verschiedene Lesemöglichkeiten für das Model später vorhanden sind. (Im Grunde werden die Enden eines Wortes abgeschnitte um zu einer Ursprungsform zu gelangen.)

Funktionsweise der Funktionen siehe 2. Tokenize.

In [26]:
def apply_stemming(tokenized_column):
  stemmer = PorterStemmer() 
  return [stemmer.stem(word) for word in tokenized_column]

In [27]:
df['porter_stemmed']=df.apply(lambda x: apply_stemming(x['stopword_removed']), axis=1)
df[['stopword_removed', 'porter_stemmed']].head

<bound method NDFrame.head of                                         stopword_removed  \
0      [schwabenhelga, natürlich, geld, möglichkeit, ...   
1      [keller, verstecken, wahre, stromfresser, heiz...   
2      [online, neunen, klimaschutzgesetz, bürgerbete...   
3      [konkrete, ausbauziele, kommenden, land, baut,...   
4      [feiern, juhubiläum, stellen, sophie, heinz, l...   
...                                                  ...   
10960                          [fixe, august, folgenden]   
10961         [fixe, august, reihenfolge, susannehennig]   
10962  [paparazzoorange, ja, fehlte, wirklich, dafür,...   
10963  [paparazzoorange, hm, ja, ja, schon, her, stim...   
10964  [tkinth, dabei, carius, gesundheitsministerin,...   

                                          porter_stemmed  
0      [schwabenhelga, natürlich, geld, möglichkeit, ...  
1      [keller, verstecken, wahr, stromfress, heizung...  
2      [onlin, neunen, klimaschutzgesetz, bürgerbetei...  
3      [konkr

# Datentyp ändern
Da wir wenig mit Listen im DF anfangen können, müssen wir die Tokenz und Listen wieder umwandeln. Wir müssen den Zusammenhang in den Zeilen (Tweets) wieder herstellen. Deshalb wandeln wir es um und trennen jeses Wort mit einem Leerzeichen.

In [28]:
def rejoined_words(tokenized_column):
  return (" ".join(tokenized_column))

In [29]:
df['rejoined']=df.apply(lambda x: rejoined_words(x['porter_stemmed']), axis=1)
df[['porter_stemmed', 'rejoined']].head

<bound method NDFrame.head of                                           porter_stemmed  \
0      [schwabenhelga, natürlich, geld, möglichkeit, ...   
1      [keller, verstecken, wahr, stromfress, heizung...   
2      [onlin, neunen, klimaschutzgesetz, bürgerbetei...   
3      [konkret, ausbauziel, kommenden, land, baut, j...   
4      [feiern, juhubiläum, stellen, sophi, heinz, li...   
...                                                  ...   
10960                           [fix, august, folgenden]   
10961           [fix, august, reihenfolg, susannehennig]   
10962  [paparazzoorang, ja, fehlt, wirklich, dafür, l...   
10963  [paparazzoorang, hm, ja, ja, schon, her, stimm...   
10964  [tkinth, dabei, cariu, gesundheitsministerin, ...   

                                                rejoined  
0      schwabenhelga natürlich geld möglichkeit dieje...  
1      keller verstecken wahr stromfress heizungspump...  
2      onlin neunen klimaschutzgesetz bürgerbeteiligu...  
3      konkre

# DataFrame wieder trennen
Für die Weiterverarbeitung müssen wir die Tweets wieder nach Ost und West trennen. Hierzu nutzen wir die zuerst eingefügten Label

In [30]:
df['Label']=pd.to_numeric(df['Label'])

In [31]:
df_westen=df[df['Label']>=1]
df_osten=df[df['Label']<1]

In [32]:
df_westen.head()

,Name,Datum,Inhalt,Label,tokenized,stopword_removed,porter_stemmed,rejoined
0,@RegierungBW,2022-09-22 07:45:03+00:00,@schwabenhelga natürlich hat nicht jeder das g...,1,"[schwabenhelga, natürlich, hat, nicht, jeder, ...","[schwabenhelga, natürlich, geld, möglichkeit, ...","[schwabenhelga, natürlich, geld, möglichkeit, ...",schwabenhelga natürlich geld möglichkeit dieje...
1,@RegierungBW,2022-09-22 07:31:26+00:00,in manchem keller verstecken sich wahre #strom...,1,"[in, manchem, keller, verstecken, sich, wahre,...","[keller, verstecken, wahre, stromfresser, heiz...","[keller, verstecken, wahr, stromfress, heizung...",keller verstecken wahr stromfress heizungspump...
2,@RegierungBW,2022-09-22 06:59:09+00:00,auf unserem #beteiligunsgsportal können sie si...,1,"[auf, unserem, beteiligunsgsportal, können, si...","[online, neunen, klimaschutzgesetz, bürgerbete...","[onlin, neunen, klimaschutzgesetz, bürgerbetei...",onlin neunen klimaschutzgesetz bürgerbeteiligu...
3,@RegierungBW,2022-09-21 11:52:04+00:00,@martinruff1 @umweltbw konkrete ausbauziele fü...,1,"[umweltbw, konkrete, ausbauziele, für, die, ko...","[konkrete, ausbauziele, kommenden, land, baut,...","[konkret, ausbauziel, kommenden, land, baut, j...",konkret ausbauziel kommenden land baut ja wind...
4,@RegierungBW,2022-09-21 11:33:38+00:00,wir feiern unser #juhubiläum mit #70jahre70köp...,1,"[wir, feiern, unser, juhubiläum, mit, heute, s...","[feiern, juhubiläum, stellen, sophie, heinz, l...","[feiern, juhubiläum, stellen, sophi, heinz, li...",feiern juhubiläum stellen sophi heinz liebt di...


In [33]:
df_osten.head()

,Name,Datum,Inhalt,Label,tokenized,stopword_removed,porter_stemmed,rejoined
0,@Brandenburg_LT,2022-09-22 09:57:00+00:00,der #petitionsausschuss des landtags #brandenb...,0,"[der, petitionsausschuss, des, landtags, brand...","[petitionsausschuss, nächste, september, komme...","[petitionsausschuss, nächste, septemb, kommen,...",petitionsausschuss nächste septemb kommen gern...
1,@Brandenburg_LT,2022-09-20 09:14:02+00:00,landtagssitzung(en) verpasst? 🫤\n\nin der @rbb...,0,"[landtagssitzung, en, verpasst, in, der, media...","[en, redebeiträge, abgeordneten, jederzeit, ru...","[en, redebeiträg, abgeordneten, jederzeit, run...",en redebeiträg abgeordneten jederzeit runtergu...
2,@Brandenburg_LT,2022-09-19 10:47:51+00:00,im gedenken an königin #elizabethii sind die f...,0,"[im, gedenken, an, königin, elizabethii, sind,...","[gedenken, königin, elizabethii, flaggen, halb...","[gedenken, königin, elizabethii, flaggen, halb...",gedenken königin elizabethii flaggen halbmast ...
3,@Brandenburg_LT,2022-09-19 08:45:15+00:00,landtagspräsidentin ulrike liedtke hat am woch...,0,"[landtagspräsidentin, ulrike, liedtke, hat, am...","[wochenende, konferenz, parlamentspräsidentinn...","[wochenend, konferenz, parlamentspräsidentinne...",wochenend konferenz parlamentspräsidentinnen e...
4,@Brandenburg_LT,2022-09-19 06:28:10+00:00,"rt @speakerpelosi: today, @g7 speakers visited...",0,"[rt, speakerpelosi, today, speakers, visited, ...","[today, speakers, visited, the, state, parliam...","[today, speaker, visit, the, state, parliament...",today speaker visit the state parliament learn...


# DataFrame herunterladen
Um die DataFrame weiterzuverwenden laden wir sie erneut als csv-Dateien herunter.

In [40]:
df_westen.to_csv(index=False)

compression_opts = dict(method='zip',
                        archive_name='westen_prepro.csv')  
df_westen.to_csv('westen_prepro.zip', index=False,
          compression=compression_opts)

In [41]:
df_osten.to_csv(index=False)

compression_opts = dict(method='zip',
                        archive_name='osten_prepro.csv')  
df_osten.to_csv('osten_prepro.zip', index=False,
          compression=compression_opts)